In [1]:
import sympy as sp
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import highspy as highs
import random
from datetime import *
import functools

In [2]:
def load_aimms(file):
    pass

In [2]:
with open(
    'Beer Transport.ams'
) as file:
    data = file.read()

lines = data.splitlines()
# sets = data.split('Set')
# parameters = sets[-1].split('Parameter')
# constraints = parameters[-1].split('Constraint')
# variables = constraints[-1].split('Variable')

# sets = sets[1:]
# sets[-1] = parameters[0]
# parameters = parameters[1:]
# parameters[-1] = constraints[0]
# constraints = constraints[1:]
# constraints[-1] = variables[0]
# variables = variables[1:]

sets_raw = []
parameters_raw = []
constraints_raw = []
variables_raw = []

n = 0
for line in lines:
    if line.startswith('\tSet'):
        end = 0
        end_line = lines[n]
        while '}' not in end_line:
            end += 1
            end_line = lines[n + end]
        sets_raw.append(lines[n : n + end])
    elif line.startswith('\tParameter'):
        end = 0
        end_line = lines[n]
        while '}' not in end_line:
            end += 1
            end_line = lines[n + end]
        parameters_raw.append(lines[n : n + end])
    elif line.startswith('\tConstraint'):
        end = 0
        end_line = lines[n]
        while '}' not in end_line:
            end += 1
            end_line = lines[n + end]
        constraints_raw.append(lines[n : n + end])
    elif line.startswith('\tVariable'):
        end = 0
        end_line = lines[n]
        while '}' not in end_line:
            end += 1
            end_line = lines[n + end]
        variables_raw.append(lines[n : n + end])
    elif line.startswith('\t\tObjective'):
        objective = line
    n += 1

sets = {}
for i in sets_raw:
    set = i[1].replace('\t\tIndex: ', '').replace(';', '')
    sets[set] = sp.Idx(set)

parameters = {}
for i in parameters_raw:
    param_name = i[0].replace('\tParameter ', '').replace(' {', '')
    param_index = (
        i[1].replace('\t\tIndexDomain: ', '').replace(';', '').replace('(', '').replace(')', '')
    )
    if len(param_index.split(',')) == 1:
        parameters[param_name] = 1
    print(param_name)
    print(param_index)

Supply
p
Demand
c
UnitTransportCost
p,c


In [4]:
sets

{'p': p, 'c': c}

In [3]:
def extract_between(text, start_char, end_char):
    start_index = text.find(start_char)
    if start_index == -1:
        return ''
    start_index += len(start_char)
    end_index = text.find(end_char, start_index)
    if end_index == -1:
        return ''
    return text[start_index:end_index]

In [54]:
region = sp.Idx('r')
year = sp.Idx('y')
len_r, len_y = sp.symbols('len_r, len_y')

max_prod = sp.IndexedBase('max_prod')
cost = sp.IndexedBase('cost')
demand = sp.IndexedBase('demand')

production = sp.IndexedBase('production')

subs = {
    'r': region,
    'y': year,
    'max_prod': max_prod,
    'cost': cost,
    'demand': demand,
    'production': production,
    'len_r': len_r,
    'len_y': len_y,
}

constraint1 = sp.parse_expr('production[r,y]', subs)

# constraint1

values = pd.read_excel('testData.xlsx').set_index(['r', 'y'])

testSum = sp.Sum(sp.Sum(constraint1, (region, 1, len_r)), (year, 1, len_y))
testSum

Sum(production[r, y], (r, 1, len_r), (y, 1, len_y))

In [8]:
testSum.subs({len_r: 10, len_y: 2}).doit()

production[1, 1] + production[1, 2] + production[10, 1] + production[10, 2] + production[2, 1] + production[2, 2] + production[3, 1] + production[3, 2] + production[4, 1] + production[4, 2] + production[5, 1] + production[5, 2] + production[6, 1] + production[6, 2] + production[7, 1] + production[7, 2] + production[8, 1] + production[8, 2] + production[9, 1] + production[9, 2]

In [5]:
eq = 'sum[ (r,y), demand(r,y)] <= sum[r, prod(r)];'.replace(';', '')


def sum_split(eq, relation):
    split_relation = eq.split(relation)
    final_sides = []
    final_sides_indices = []
    for side in split_relation:
        if 'sum' not in side:
            final_sides.append(side)
            final_sides_indices.append('')
        else:
            temp = extract_between(side, '[', ']').strip()
            if temp[0] == '(':
                indices = extract_between(temp, '(', ')')
                # print(indices)
                exp = temp.split('),')[1].strip().replace('(', '[').replace(')', ']')
                # print(exp)
            else:
                indices = temp.split(',')[0].strip()
                exp = temp.split(',')[1].strip().replace('(', '[').replace(')', ']')
                # print(indices)
                # print(exp)
            final_sides.append(exp)
            final_sides_indices.append(indices)
    return final_sides, final_sides_indices


def aimms_to_python(exp: str):
    exp = exp.replace('(', '[')
    exp = exp.replace(')', ']')
    exp = exp.split(';')[0]
    return exp


# if 'sum' in eq:
def parse_expr_sum(eq: str, subs):
    eq = eq.split(';')[0]
    relation = ''
    if '<=' in eq:
        relation = '<='
    elif '>=' in eq:
        relation = '>='
    elif '!=' in eq:
        relation = '!='
    elif '<' in eq:
        relation = '<'
    elif '>' in eq:
        relation = '>'
    elif '==' in eq:
        relation = '=='
    else:
        print('something wrong')
        return

    split_relation = eq.split(relation)
    final_sides = []
    final_sides_indices = []
    indices_lengths = {}
    for side in split_relation:
        if 'sum' not in side:
            final_sides.append(sp.parse_expr(aimms_to_python(side), subs))
            final_sides_indices.append('')
        else:
            temp = extract_between(side, '[', ']').strip()
            if temp[0] == '(':
                indices = extract_between(temp, '(', ')')
                # indices = sp.symbols(indices)
                indices = indices.split(',')
                for i in indices:
                    indices_lengths[i] = sp.Symbol(f'len_{i}')
                    # i = subs[i]
                exp = temp.split('),')[1].strip().replace('(', '[').replace(')', ']')
                exp = sp.parse_expr(exp, subs)
            else:
                indices = temp.split(',')[0].strip()
                indices_lengths = {indices[0]: sp.Symbol(f'len_{indices[0]}')}
                # indices = [subs[indices]]
                exp = temp.split(',')[1].strip().replace('(', '[').replace(')', ']')
                exp = sp.parse_expr(exp, subs)
            for index in indices:
                exp = sp.Sum(exp, (subs[index], 1, indices_lengths[index]))
            final_sides.append(exp)
            final_sides_indices.append(indices)

    return sp.Rel(final_sides[0], final_sides[1], relation)

    # print(final_sides_indices)
    # print(final_sides)

    # con = sp.parse_expr(final_sides[0],subs)
    # new_sum = sp.Sum(con, (final_sides_indices[0].split(',')[0],'a','b'))
    # new_sum

In [ ]:
testEq = parse_expr_sum(eq, subs)
testEq

In [6]:
# load data
set_values = {}
parameter_values = {}

values = pd.read_excel('testData.xlsx')
r_values = values.r.unique()
y_values = values.y.unique()
index_map = {
    (i, j): (r_values[i - 1], y_values[j - 1])
    for i in range(1, len(r_values) + 1)
    for j in range(1, len(y_values) + 1)
}
demand_df = values.set_index(['r', 'y'])

In [156]:
# testEq.subs({len_r:10})
# testEq.subs({demand[first,first]:0, last:len(r_len)}).doit()


# testEq.subs({demand[r,y]:demand_df[(r,y)] for r in range(1,11) for y in range(2020,2022)})

sub_test = testEq.subs({len_r: len(r_values), len_y: len(y_values)}).doit()
for i, j in index_map.keys():
    # print((i,j))
    # print(index_map[(i,j)])
    sub_test = sub_test.subs({demand[i, j]: demand[index_map[(i, j)]]})

sub_test

demand[1, 2020] + demand[1, 2021] + demand[10, 2020] + demand[10, 2021] + demand[2, 2020] + demand[2, 2021] + demand[3, 2020] + demand[3, 2021] + demand[4, 2020] + demand[4, 2021] + demand[5, 2020] + demand[5, 2021] + demand[6, 2020] + demand[6, 2021] + demand[7, 2020] + demand[7, 2021] + demand[8, 2020] + demand[8, 2021] + demand[9, 2020] + demand[9, 2021] <= prod[10] + prod[1] + prod[2] + prod[3] + prod[4] + prod[5] + prod[6] + prod[7] + prod[8] + prod[9]

In [7]:
sets = ['r', 'y']
parameters = ['max_prod', 'cost', 'demand']
variables = ['production']
# constraints = ['max_prod', 'total_demand', 'non_negative']

In [8]:
max_prod_con = 'production(r,y) <= max_prod(r,y);'

total_demand = 'sum[ (r,y), demand(r,y)] <= sum[ (r,y), production(r,y)];'

non_negative = 'production(r,y) >= 0;'
constraints = {
    'max_prod_constraint': max_prod_con,
    'total_demand': total_demand,
    'non_negative': non_negative,
}

In [ ]:
"""mod = pyo.ConcreteModel()
bimap = {}
for set in sets:
    exec(f'mod.{set} = pyo.Set(initialize={set}_values)')
    bimap[subs[set]] = exec(f'mod.{set}')
    # exec(f'bimap[mod.{set}] = "{subs[set]}"')
for p in parameters:
    temp_dict = {(i,j):demand_df[p][(i,j)] for (i,j) in demand_df.index}
    indices = ''
    for s in sets:
        indices += f'mod.{s},'
    exec(f'mod.{p} = pyo.Param({indices} mutable=True, initialize={temp_dict})')
    bimap[subs[p]] = exec(f'mod.{p}')
for v in variables:
    indices = ''
    for s in sets:
        indices += f'mod.{s},'
    exec(f'mod.{v} = pyo.Var({indices})')
    bimap[subs[v]] = exec(f'mod.{v}')
for p in mod.component_objects(pyo.Param):
    for i in p:
        bimap[subs[p.name][i]] = p[i]
for v in mod.component_objects(pyo.Var):
    for i in v:
        bimap[subs[v.name][i]] = v[i]
s2pBimap = pyo.sympy_tools.PyomoSympyBimap()
s2pBimap.sympy2pyomo = bimap
subs[len_r] = len(r_values)
subs[len_y] = len(y_values)
exps = {}
for c in constraints.keys():
    if 'sum' in constraints[c]:
        exp = parse_expr_sum(constraints[c],subs).subs({subs['len_y']:len(y_values), subs['len_r']:len(r_values)}).doit()
    else:
        exp = sp.parse_expr(aimms_to_python(constraints[c]),subs)
    exps[c] = exp
    # @mod.Constraint(mod.r, mod.y)
    # def con(mod, r, y):
    #     # s2pBimap.sympy2pyomo[subs[r]] = r
    #     # s2pBimap.sympy2pyomo[subs[y]] = y
    #     pyoExp = pyo.sympy_tools.sympy2pyomo_expression(exp,s2pBimap)
    #     return pyoExp
    
# def c_function(mod, r, y):
#     constraint = sp.lambdify([subs['production'][subs['r'],subs['y']], subs['max_prod'][subs['r'],subs['y']]],exps['max_prod_constraint'])
#     return constraint(r, y)
# mod.max_prod_constraint = pyo.Constraint(mod.r, mod.y, rule=c_function)

# @mod.Constraint(mod.r, mod.y)
# def max_prod_constraint(mod, r, y):
#     exp_sub = exps['max_prod_constraint'].subs({subs['r']:r, subs['y']:y})
#     s2pBimap.sympy2pyomo[subs['production'][r,y]] = mod.production[r,y]
#     s2pBimap.sympy2pyomo[subs['max_prod'][r,y]] = mod.max_prod[r,y]
#     expression = pyo.sympy_tools.sympy2pyomo_expression(exps['max_prod_constraint'], s2pBimap)
#     return expression

##################################################################################################
#Unfinished Project
#Couldn't get indexed objects to convert to pyomo correctly
##################################################################################################

mod.pprint()"""

2 Set Declarations
    r : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    y : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {2020, 2021}

3 Param Declarations
    cost : Size=20, Index=r*y, Domain=Any, Default=None, Mutable=True
        Key        : Value
         (1, 2020) :  5.79296757323126
         (1, 2021) : 7.234093446155088
         (2, 2020) : 7.055929592835262
         (2, 2021) : 7.028925755382812
         (3, 2020) : 7.561123627149676
         (3, 2021) : 5.788958639306633
         (4, 2020) : 5.716360940091986
         (4, 2021) : 5.204751479645354
         (5, 2020) :  7.13064158129915
         (5, 2021) : 9.423005853800866
         (6, 2020) :  8.52961118862886
         (6, 2021) : 7.782647301897869
         (7, 2020) : 6.160652601532298
         (7, 2021) :  6.48889495495775
       

In [75]:
# keys = {}
# for k in exp.free_symbols:
#     keys[str(k)] = k
# exp.subs({keys['r']:10, keys['y']:2}).free_symbols

pyo.sympy_tools.sympy2pyomo_expression(exps['max_prod_constraint'], s2pBimap)

TypeError: Cannot convert expression to float

In [15]:
class aimms_sensitivity:
    def __init__(self, ams, sets, parameters, parsed=False):
        pass

    def parse_aimms(ams):
        with open(ams) as file:
            data = file.read()
        lines = data.splitlines()
        sets_raw = []
        parameters_raw = []
        constraints_raw = []
        variables_raw = []

        n = 0
        for line in lines:
            if line.startswith('\tSet'):
                end = 0
                end_line = lines[n]
                while '}' not in end_line:
                    end += 1
                    end_line = lines[n + end]
                sets_raw.append(lines[n : n + end])
            elif line.startswith('\tParameter'):
                end = 0
                end_line = lines[n]
                while '}' not in end_line:
                    end += 1
                    end_line = lines[n + end]
                parameters_raw.append(lines[n : n + end])
            elif line.startswith('\tConstraint'):
                end = 0
                end_line = lines[n]
                while '}' not in end_line:
                    end += 1
                    end_line = lines[n + end]
                constraints_raw.append(lines[n : n + end])
            elif line.startswith('\tVariable'):
                end = 0
                end_line = lines[n]
                while '}' not in end_line:
                    end += 1
                    end_line = lines[n + end]
                variables_raw.append(lines[n : n + end])
            elif line.startswith('\t\tObjective'):
                objective = line
            n += 1
        return sets_raw, parameters_raw, constraints_raw, variables_raw, objective

    def extract_between(self, text, start_char, end_char):
        start_index = text.find(start_char)
        if start_index == -1:
            return ''
        start_index += len(start_char)
        end_index = text.find(end_char, start_index)
        if end_index == -1:
            return ''
        return text[start_index:end_index]

    def parse_expr_sum(self, eq: str, subs):
        relation = ''
        if '<=' in eq:
            relation = '<='
        elif '>=' in eq:
            relation = '>='
        elif '!=' in eq:
            relation = '!='
        elif '<' in eq:
            relation = '<'
        elif '>' in eq:
            relation = '>'
        elif '==' in eq:
            relation = '=='
        else:
            print('something wrong')
            return

        first, last = sp.symbols('first, last')
        split_relation = eq.split(relation)
        final_sides = []
        final_sides_indices = []
        for side in split_relation:
            if 'sum' not in side:
                final_sides.append(sp.parse_expr(side, subs))
                final_sides_indices.append('')
            else:
                temp = extract_between(side, '[', ']').strip()
                if temp[0] == '(':
                    indices = extract_between(temp, '(', ')')
                    indices = sp.symbols(indices)
                    exp = temp.split('),')[1].strip().replace('(', '[').replace(')', ']')
                    exp = sp.parse_expr(exp, subs)
                else:
                    indices = temp.split(',')[0].strip()
                    indices = [sp.symbols(indices)]
                    exp = temp.split(',')[1].strip().replace('(', '[').replace(')', ']')
                    exp = sp.parse_expr(exp, subs)
                for index in indices:
                    exp = sp.Sum(exp, (index, first, last))
                final_sides.append(exp)
                final_sides_indices.append(indices)

        return sp.Rel(final_sides[0], final_sides[1], relation)

    @classmethod
    def create_from_file(cls, ams, file):
        cls.parse_aimms(file)

In [41]:
a, b, x, y = sp.symbols('a, b, x, y')
exp1 = a * x + b * y <= 0
mod1 = pyo.ConcreteModel()
mod1.a = pyo.Param(initialize=20)
mod1.b = pyo.Param(initialize=10)
mod1.x = pyo.Var()
mod1.y = pyo.Var()

newBimap = pyo.sympy_tools.PyomoSympyBimap()
newBimap.sympy2pyomo = {a: mod1.a, b: mod1.b, x: mod1.x, y: mod1.y}

exp1b = pyo.sympy_tools.sympy2pyomo_expression(exp1, newBimap)

mod1.set1 = pyo.Set(initialize=[1, 2, 3, 4, 5])
mod1.param1 = pyo.Param(mod1.set1, initialize={1: 10, 2: 11, 3: 12, 4: 13, 5: 14})

for p in mod1.component_objects(pyo.Param):
    print(type(p.name))

<class 'str'>
<class 'str'>
<class 'str'>
